In [67]:
import pandas as pd
import numpy as np

we are loading the csv files (heartrate, activity-step counts and sleep)

In [68]:
heartrate = pd.read_csv("heartrate_seconds_merged.csv", header=1)
activity = pd.read_csv("dailyactivity_merged.csv", header=1)
sleep = pd.read_csv("sleepDay_merged.csv", header=1)
heartrate.head()
activity.head()
sleep.head()

,Id,SleepDay,TotalSleepRecords,TotalMinutesAsleep,TotalTimeInBed
0,1503960366,4/12/2016 12:00:00 AM,1,327,346
1,1503960366,4/13/2016 12:00:00 AM,2,384,407
2,1503960366,4/15/2016 12:00:00 AM,1,412,442
3,1503960366,4/16/2016 12:00:00 AM,2,340,367
4,1503960366,4/17/2016 12:00:00 AM,1,700,712


normalizing the timestamps to utc

In [69]:
heartrate['Time'] = pd.to_datetime(heartrate['Time'], utc=True)

activity['ActivityDate'] = pd.to_datetime(activity['ActivityDate'], utc=True)

sleep['SleepDay'] = pd.to_datetime(sleep['SleepDay'], utc=True)

/tmp/ipython-input-2615299105.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  sleep['SleepDay'] = pd.to_datetime(sleep['SleepDay'], utc=True)


checking for the missing values for the given csv files.(so whenever we encounter 0 it would be converted to boolean value true )

In [70]:
heartrate.isnull().sum()
activity.isnull().sum()
sleep.isnull().sum()
heartrate.fillna(method='ffill', inplace=True)
activity.fillna(0, inplace=True)
sleep.fillna(0, inplace=True)

/tmp/ipython-input-3521399508.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  heartrate.fillna(method='ffill', inplace=True)


we are converting the heart rate per seconds into 1 single time interval and computing the average heart rates within that time frame

In [71]:
heartrate_interval = (
 heartrate.set_index(['Id', 'Time'])
  .groupby(level='Id')
  .resample('1min', level='Time')
  .mean()
  .reset_index()
 )

merging to same column names

In [72]:
activity.rename(columns={'ActivityDate': 'Time'}, inplace=True)
sleep.rename(columns={'SleepDay': 'Time'}, inplace=True)

sorted columns

In [73]:
merged = pd.merge(heartrate_interval, activity, on=['Id', 'Time'], how='left')
merged = pd.merge(merged, sleep, on=['Id', 'Time'], how='left')
merged.fillna(0, inplace=True)
merged.sort_values(by='Time', inplace=True)
merged.head()

,Id,Time,Value,TotalSteps,TotalDistance,TrackerDistance,LoggedActivitiesDistance,VeryActiveDistance,ModeratelyActiveDistance,LightActiveDistance,SedentaryActiveDistance,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories,TotalSleepRecords,TotalMinutesAsleep,TotalTimeInBed
100706,4020332650,2016-04-12 00:00:00+00:00,63.500000,8539.0,6.12,6.12,0.0,0.15,0.24,5.68,0.0,4.0,15.0,331.0,712.0,3654.0,1.0,501.0,541.0
100707,4020332650,2016-04-12 00:01:00+00:00,65.913043,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100708,4020332650,2016-04-12 00:02:00+00:00,67.000000,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100709,4020332650,2016-04-12 00:03:00+00:00,67.000000,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100710,4020332650,2016-04-12 00:04:00+00:00,67.000000,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [75]:
merged.to_csv("Clean.csv", index=False)